In [44]:
import pandas as pd
import keras
import seaborn as sns
import matplotlib.pyplot as plt
import os

data = pd.read_csv("unemployment-rate-men-vs-women.csv")

In [2]:
data.columns

Index(['Entity', 'Code', 'Year',
       'Unemployment, female (% of female labor force) (modeled ILO estimate)',
       'Unemployment, male (% of male labor force) (modeled ILO estimate)',
       'Population (historical estimates)', 'Continent'],
      dtype='object')

In [3]:
mapper = {'Unemployment, female (% of female labor force) (modeled ILO estimate)': 'unemployment_women',
          'Unemployment, male (% of male labor force) (modeled ILO estimate)': 'unemployment_men',
          'Population (historical estimates)': 'population'
          }

data.rename(columns=mapper, inplace=True)
data.rename(str.lower, axis='columns', inplace=True)


In [4]:
data.isna().sum()

type(data)

pandas.core.frame.DataFrame

In [13]:
# data['code'].sort_values('code').fillna(method='ffill', inplace=True)
# data['unemployment_men'].fillna(value=data['unemployment_men'].mean(), inplace=True)
# data['unemployment_women'].fillna(value=data['unemployment_women'].mean(), inplace=True)
# data['population'].fillna(value=data['population'].mean(), inplace=True)

def filler(dataframe: pd.DataFrame, column: str):
    """
    applies the forward fill method of the pandas package on a column in a dataframe

    ARGS:
    """
    dataframe[column] = dataframe[column].fillna(method='ffill')
    return dataframe[column]

def mean_filler(dataframe: pd.DataFrame, column: str):
    """
    applies mean of a column on all missing values in a column, do not use of missing values are too much
    """
    dataframe[column] = dataframe[column].fillna(value=dataframe[column].mean())
    return dataframe[column]

filler(data, "code")
filler(data, 'continent')
mean_filler(data, 'unemployment_men')
mean_filler(data, 'unemployment_women')
mean_filler(data, 'population')

data.to_csv("unemployment_rate.csv")

C:\Users\KingED\AppData\Local\Temp\ipykernel_22936\763551472.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataframe[column] = dataframe[column].fillna(method='ffill')


In [43]:
new = pd.read_csv("unemployment_rate.csv")

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoder = LabelEncoder()
hotcoder = OneHotEncoder()
new['continent'] = encoder.fit_transform(new['continent'])

x =new[['continent', 'year', 'unemployment_women', 'population']]
y =new['unemployment_men']
input_shape = len(x.columns)
print(input_shape, new.shape[1])

model = Sequential()
model.add(Dense(100, input_shape=(4, ), activation='linear'))
model.add(Dense(75, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics='accuracy')
model.fit(x, y, epochs=15)


4 8
Epoch 1/15
1831/1831 [==============================] - 8s 4ms/step - loss: 43758649344000.0000 - accuracy: 1.7071e-05
Epoch 2/15
1831/1831 [==============================] - 6s 3ms/step - loss: 4913079779328.0000 - accuracy: 1.7071e-05
Epoch 3/15
1831/1831 [==============================] - 6s 3ms/step - loss: 487909.9688 - accuracy: 0.0000e+00
Epoch 4/15
1831/1831 [==============================] - 7s 4ms/step - loss: 1265532665856.0000 - accuracy: 0.0000e+00
Epoch 5/15
1831/1831 [==============================] - 6s 3ms/step - loss: 396.2180 - accuracy: 1.7071e-05
Epoch 6/15
1831/1831 [==============================] - 6s 3ms/step - loss: 100651663360.0000 - accuracy: 0.0000e+00
Epoch 7/15
1831/1831 [==============================] - 6s 3ms/step - loss: 38706.2891 - accuracy: 0.0000e+00
Epoch 8/15
1831/1831 [==============================] - 7s 4ms/step - loss: 66.1365 - accuracy: 0.0000e+00
Epoch 9/15
1831/1831 [==============================] - 6s 3ms/step - loss: 70.8197 - ac

In [ ]:
# hyperparameter tuning
# - you can use batch normalization from the keras.layers module
# - you can convert keras model to scikitlearn estimator using the scikeras module, [from scikeras import KerasClassifier]

# random search on keras

# model = Sequential([Dense(input_sape=(), activation='relu', units=100),
#                     Dense(activation='relu', units=50),
#                     Dense(activation='softmax', units=10)])

# model_sk = KerasClassifier(build_fn=model, batch_size=x, epochs=y)

# params = {'optimizer': ['sdg', 'adama'],
#           'epochs': 3,
#           'activation': ['relu', 'tanh'],}

# random = RandomSearchCV(model_sk, params_dict=params, cv=3)
# randomised = random.fit(x, y)

# print(f"Best score is {randomised.best_score_},
#        using {randomised.best_params_}")

# from typing import List

# def model(nn: List[int], nl: List[int]):
#     model = Sequential()
#     model.add(Dense(16, input_shape=(2, ), activation='relu'))
#     for i in range(nl):
#        model.add(nn, activation='relu')
#        model.add(2, activation='sigmoid')
#        model.compile(optimizer='adam', loss='mse')

# model